In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 15)})

from tqdm.notebook import tqdm
tqdm().pandas()

from scipy.stats import ttest_rel

acousticbrainz = pd.read_hdf(Path.cwd() / 'datasets' / 'acousticbrainzV3.h5')
spotify = pd.read_hdf(Path.cwd() / 'datasets' / 'spotify.h5')
mapping = pd.read_hdf(Path.cwd() / 'datasets' / 'mapping.h5')

def findpairs(merged, bitrate, codecs):
    # Resulting dataframes
    res = {}
    for codec in codecs:
        res[codec] = []
    
    # Filter on bitrate
    filtered = merged[merged['bit_rate'] == bitrate]
    
    # Within this bitrate, try to find pairs of the same MBID with different codecs
    
    # Must have at least two submissions
    grouped = filtered.groupby('mbid').filter(lambda x: len(x) > 1)
    
    for name, group in tqdm(grouped.groupby('mbid')):
        for i in range(0, len(group)-1):
            if group.iloc[i]['codec'] in codecs:
                codec_a = group.iloc[i]['codec']
                
                for j in range(i+1, len(group)):
                    if group.iloc[j]['codec'] in codecs:
                        codec_b = group.iloc[j]['codec']
                        
                        # Pair found!
                        if codec_a != codec_b:
                            res[codec_a].append(group.iloc[i])
                            res[codec_b].append(group.iloc[j])
    
    return res

def significancetest(series_a, series_b, col_1, col_2):
    assert(len(series_a) == len(series_b))
    
    a = []
    b = []
    
    for i in range(0, len(series_a)):
        a.append(abs(series_a[i][col_1] - series_a[i][col_2]))
        b.append(abs(series_b[i][col_1] - series_b[i][col_2]))
    
    statistic, p = ttest_rel(a, b)
    return (np.mean(a), np.mean(b), statistic, p)
                        
                        
            

C:\Users\Chris\Miniconda3\envs\thesis2\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
acousticbrainz['mbid'] = acousticbrainz.index.get_level_values(0)
merged = acousticbrainz.merge(mapping, on='mbid').merge(spotify, left_on='spotify', right_on='id')

In [3]:
onesixty = findpairs(merged, 160000, ['mp3', 'vorbis'])
oneninetytwo = findpairs(merged, 192000, ['mp3', 'vorbis'])
twofiftysix = findpairs(merged, 256000, ['mp3', 'vorbis'])
threetwenty = findpairs(merged, 320000, ['mp3', 'vorbis'])

In [4]:
dfs = [
    onesixty,
    oneninetytwo,
    twofiftysix,
    threetwenty
]

samplesizes = {
    160000: len(onesixty['mp3']),
    192000: len(oneninetytwo['mp3']),
    256000: len(twofiftysix['mp3']),
    320000: len(threetwenty['mp3'])
}

cols = [
    ('acousticness', ('mood_acoustic', 'acoustic')),
    ('danceability', ('danceability', 'danceable')),
    ('energy', ('mood_relaxed', 'not_relaxed')),
    ('instrumentalness', ('voice_instrumental', 'instrumental')),
    ('valence', ('mood_happy', 'happy')),
    ('valence', ('mood_sad', 'not_sad'))
]

res_all = {}

for rate in [160000, 192000, 256000, 320000]:
    res_all[rate] = pd.DataFrame()

for df in dfs:
    res = pd.DataFrame()
    
    codec_a = list(df.keys())[0]
    codec_b = list(df.keys())[1]
    bitrate = df[list(df.keys())[0]][0]['bit_rate']
    
    for col in cols:
        ttest = significancetest(df[codec_a], df[codec_b], col[0], col[1])
        toadd = pd.Series(data=[ttest[0], ttest[1], ttest[2], ttest[3]], name=col)
        res = res.append(toadd)
    
    res.columns = pd.MultiIndex.from_tuples([('MAE', codec_a), ('MAE', codec_b), ('t',''), ('p','')])
    res_all[bitrate] = res

In [6]:
for key, value in res_all.items():
    print(f"Bitrate: {key}, n={samplesizes[key]}")
    display(value)
    print(value.to_latex())

Bitrate: 160000, n=19


MAE            \
                                                         mp3    vorbis   
(acousticness, (mood_acoustic, acoustic))           0.215574  0.192192   
(danceability, (danceability, danceable))           0.439985  0.326822   
(energy, (mood_relaxed, not_relaxed))               0.286043  0.286476   
(instrumentalness, (voice_instrumental, instrum...  0.600029  0.474101   
(valence, (mood_happy, happy))                      0.202633  0.356557   
(valence, (mood_sad, not_sad))                      0.377937  0.388755   

                                                           t         p  
                                                                        
(acousticness, (mood_acoustic, acoustic))           0.231516  0.819526  
(danceability, (danceability, danceable))           3.110071  0.006045  
(energy, (mood_relaxed, not_relaxed))              -0.007000  0.994492  
(instrumentalness, (voice_instrumental, instrum...  0.986925  0.336761  
(valence, (mood_happy, happy))                     -1.910047  0.072193  
(valence, (mood_sad, not_sad))                     -0.171376  0.865840

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{MAE} &         t &         p \\
{} &       mp3 & \multicolumn{3}{l}{vorbis} \\
\midrule
(acousticness, (mood\_acoustic, acoustic))          &  0.215574 &  0.192192 &  0.231516 &  0.819526 \\
(danceability, (danceability, danceable))          &  0.439985 &  0.326822 &  3.110071 &  0.006045 \\
(energy, (mood\_relaxed, not\_relaxed))              &  0.286043 &  0.286476 & -0.007000 &  0.994492 \\
(instrumentalness, (voice\_instrumental, instrum... &  0.600029 &  0.474101 &  0.986925 &  0.336761 \\
(valence, (mood\_happy, happy))                     &  0.202633 &  0.356557 & -1.910047 &  0.072193 \\
(valence, (mood\_sad, not\_sad))                     &  0.377937 &  0.388755 & -0.171376 &  0.865840 \\
\bottomrule
\end{tabular}

Bitrate: 192000, n=299


MAE            \
                                                         mp3    vorbis   
(acousticness, (mood_acoustic, acoustic))           0.172869  0.193616   
(danceability, (danceability, danceable))           0.459768  0.526277   
(energy, (mood_relaxed, not_relaxed))               0.411277  0.463551   
(instrumentalness, (voice_instrumental, instrum...  0.578092  0.587861   
(valence, (mood_happy, happy))                      0.369526  0.398368   
(valence, (mood_sad, not_sad))                      0.278248  0.256190   

                                                           t         p  
                                                                        
(acousticness, (mood_acoustic, acoustic))          -2.143873  0.032852  
(danceability, (danceability, danceable))          -4.713185  0.000004  
(energy, (mood_relaxed, not_relaxed))              -3.617660  0.000349  
(instrumentalness, (voice_instrumental, instrum... -0.441484  0.659183  
(valence, (mood_happy, happy))                     -2.418134  0.016200  
(valence, (mood_sad, not_sad))                      2.281263  0.023238

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{MAE} &         t &         p \\
{} &       mp3 & \multicolumn{3}{l}{vorbis} \\
\midrule
(acousticness, (mood\_acoustic, acoustic))          &  0.172869 &  0.193616 & -2.143873 &  0.032852 \\
(danceability, (danceability, danceable))          &  0.459768 &  0.526277 & -4.713185 &  0.000004 \\
(energy, (mood\_relaxed, not\_relaxed))              &  0.411277 &  0.463551 & -3.617660 &  0.000349 \\
(instrumentalness, (voice\_instrumental, instrum... &  0.578092 &  0.587861 & -0.441484 &  0.659183 \\
(valence, (mood\_happy, happy))                     &  0.369526 &  0.398368 & -2.418134 &  0.016200 \\
(valence, (mood\_sad, not\_sad))                     &  0.278248 &  0.256190 &  2.281263 &  0.023238 \\
\bottomrule
\end{tabular}

Bitrate: 256000, n=74


MAE            \
                                                         mp3    vorbis   
(acousticness, (mood_acoustic, acoustic))           0.180786  0.177941   
(danceability, (danceability, danceable))           0.326517  0.331549   
(energy, (mood_relaxed, not_relaxed))               0.238797  0.240557   
(instrumentalness, (voice_instrumental, instrum...  0.346797  0.313191   
(valence, (mood_happy, happy))                      0.245510  0.235400   
(valence, (mood_sad, not_sad))                      0.372572  0.376357   

                                                           t         p  
                                                                        
(acousticness, (mood_acoustic, acoustic))           0.129520  0.897302  
(danceability, (danceability, danceable))          -0.346482  0.729977  
(energy, (mood_relaxed, not_relaxed))              -0.118638  0.905888  
(instrumentalness, (voice_instrumental, instrum...  1.095717  0.276804  
(valence, (mood_happy, happy))                      0.550763  0.583478  
(valence, (mood_sad, not_sad))                     -0.202600  0.840012

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{MAE} &         t &         p \\
{} &       mp3 & \multicolumn{3}{l}{vorbis} \\
\midrule
(acousticness, (mood\_acoustic, acoustic))          &  0.180786 &  0.177941 &  0.129520 &  0.897302 \\
(danceability, (danceability, danceable))          &  0.326517 &  0.331549 & -0.346482 &  0.729977 \\
(energy, (mood\_relaxed, not\_relaxed))              &  0.238797 &  0.240557 & -0.118638 &  0.905888 \\
(instrumentalness, (voice\_instrumental, instrum... &  0.346797 &  0.313191 &  1.095717 &  0.276804 \\
(valence, (mood\_happy, happy))                     &  0.245510 &  0.235400 &  0.550763 &  0.583478 \\
(valence, (mood\_sad, not\_sad))                     &  0.372572 &  0.376357 & -0.202600 &  0.840012 \\
\bottomrule
\end{tabular}

Bitrate: 320000, n=1203


MAE            \
                                                         mp3    vorbis   
(acousticness, (mood_acoustic, acoustic))           0.161619  0.125306   
(danceability, (danceability, danceable))           0.439657  0.306358   
(energy, (mood_relaxed, not_relaxed))               0.424091  0.270790   
(instrumentalness, (voice_instrumental, instrum...  0.551969  0.294039   
(valence, (mood_happy, happy))                      0.307287  0.283616   
(valence, (mood_sad, not_sad))                      0.353860  0.320952   

                                                            t             p  
                                                                             
(acousticness, (mood_acoustic, acoustic))            6.333075  3.388418e-10  
(danceability, (danceability, danceable))           19.978805  6.621738e-77  
(energy, (mood_relaxed, not_relaxed))               18.942335  3.153612e-70  
(instrumentalness, (voice_instrumental, instrum...  21.173780  8.065342e-85  
(valence, (mood_happy, happy))                       3.819138  1.407557e-04  
(valence, (mood_sad, not_sad))                       6.202881  7.612269e-10

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{MAE} &          t &             p \\
{} &       mp3 & \multicolumn{3}{l}{vorbis} \\
\midrule
(acousticness, (mood\_acoustic, acoustic))          &  0.161619 &  0.125306 &   6.333075 &  3.388418e-10 \\
(danceability, (danceability, danceable))          &  0.439657 &  0.306358 &  19.978805 &  6.621738e-77 \\
(energy, (mood\_relaxed, not\_relaxed))              &  0.424091 &  0.270790 &  18.942335 &  3.153612e-70 \\
(instrumentalness, (voice\_instrumental, instrum... &  0.551969 &  0.294039 &  21.173780 &  8.065342e-85 \\
(valence, (mood\_happy, happy))                     &  0.307287 &  0.283616 &   3.819138 &  1.407557e-04 \\
(valence, (mood\_sad, not\_sad))                     &  0.353860 &  0.320952 &   6.202881 &  7.612269e-10 \\
\bottomrule
\end{tabular}

